# Procesamiento del Lenguaje Natural
## Asistente NLP con Docker, n8n y Ollama

**Materia:** Procesamiento del Lenguaje Natural e Introducción a LLMs  
**Módulo:** Infraestructura y Despliegue de Aplicaciones NLP

---

## Introducción

En esta guía vas a aprender a construir un sistema completo de procesamiento de lenguaje natural usando herramientas de código abierto. El objetivo es que entiendas cómo funcionan las aplicaciones de NLP en entornos reales, más allá de los notebooks de Jupyter.

### ¿Qué vamos a construir?

Un asistente de lenguaje natural que puede realizar múltiples tareas:

- **Named Entity Recognition (NER):** Extracción de entidades nombradas
- **Análisis de Sentimiento:** Clasificación emocional de textos
- **Sumarización:** Generación de resúmenes automáticos
- **Clasificación de Textos:** Identificación de tipo y tema
- **Extracción de Keywords:** Términos más relevantes

### Stack Tecnológico

- **Docker + Docker Compose:** Containerización y orquestación
- **n8n:** Plataforma de automatización de workflows
- **Ollama:** Servidor para ejecutar modelos de lenguaje localmente
- **IBM Granite 4:** Modelo de lenguaje de última generación
- **Streamlit:** Framework para crear interfaces web (opcional)

### Objetivos de Aprendizaje

Al finalizar este laboratorio vas a poder:

1. Comprender qué es Docker y por qué se usa en proyectos de IA
2. Configurar y orquestar servicios con Docker Compose
3. Trabajar con modelos de lenguaje locales sin depender de APIs externas
4. Crear workflows de NLP de forma visual con n8n
5. Implementar un asistente multimodal de procesamiento de lenguaje
6. Crear imágenes Docker personalizadas
7. Entender los fundamentos del despliegue en servidores

### Nota Importante sobre el Entorno de Ejecución

**Este proyecto NO puede ejecutarse en Google Colab** porque requiere Docker, que no está disponible en entornos Jupyter en la nube. Vas a necesitar trabajar en tu máquina local con Docker Desktop instalado.

Este notebook funciona como una **guía de referencia** que vas a consultar mientras trabajás en tu terminal.

---

## Conceptos Fundamentales

Antes de comenzar con la implementación, es fundamental que entiendas los conceptos detrás de las tecnologías que vamos a usar.

### Docker: Containerización de Aplicaciones

Docker es una plataforma que permite empaquetar aplicaciones con todas sus dependencias en **contenedores**. Un contenedor es como una caja aislada que contiene:

- El código de tu aplicación
- Las librerías y dependencias necesarias
- Configuraciones del sistema
- Un mini sistema operativo base

**¿Por qué es importante Docker?**

El problema clásico en desarrollo de software es: "en mi máquina funciona". Docker lo resuelve garantizando que tu aplicación corra exactamente igual en cualquier lugar: tu laptop, el servidor de producción, la máquina de un colega.

**Ventajas:**
- **Portabilidad:** El mismo contenedor corre en Windows, Mac, Linux
- **Reproducibilidad:** Mismo entorno siempre
- **Aislamiento:** Cada servicio en su propio contenedor
- **Eficiencia:** Los contenedores comparten el kernel del sistema operativo, son más livianos que máquinas virtuales

### Docker Compose: Orquestación de Servicios

Docker Compose es una herramienta para definir y ejecutar aplicaciones Docker con múltiples contenedores. En lugar de levantar cada contenedor manualmente, definís todos los servicios en un archivo YAML y los levantás con un solo comando.

En nuestro proyecto vamos a orquestar dos servicios:
- **n8n:** El motor de workflows
- **Ollama:** El servidor de modelos de lenguaje

### n8n: Automatización Visual de Workflows

n8n ("node-to-node") es una plataforma open source de automatización que te permite crear flujos de trabajo conectando diferentes servicios. Es similar a herramientas como Zapier o Make, pero autohospedada y con más control.

**¿Por qué usar n8n?**

Podríamos escribir todo el código en Python, pero n8n nos da:
- **Visualización:** Ves el flujo de datos gráficamente
- **Debugging:** Podés ejecutar paso a paso y ver resultados intermedios
- **Flexibilidad:** Fácil de modificar sin tocar código
- **Integraciones:** Conectores pre-construidos para cientos de servicios

En nuestro proyecto, n8n va a:
1. Recibir consultas del usuario vía webhook
2. Detectar qué tipo de tarea de NLP quiere realizar
3. Construir el prompt apropiado para el modelo
4. Enviar la solicitud a Ollama
5. Formatear y devolver la respuesta

### Ollama: Ejecución Local de Modelos de Lenguaje

Ollama es una herramienta que facilita la ejecución de modelos de lenguaje grandes (LLMs) en tu propia máquina. Maneja automáticamente la descarga, configuración y ejecución de modelos.

**Ventajas de LLMs locales:**
- **Sin costos por uso:** No pagás por cada llamada a la API
- **Privacidad:** Tus datos no salen de tu infraestructura
- **Sin límites de rate:** No hay restricciones de requests por minuto
- **Funciona offline:** No dependés de conexión a internet

**Desventajas:**
- Requiere recursos computacionales (RAM, CPU/GPU)
- Los modelos ocupan espacio en disco
- Generalmente son menos potentes que los modelos comerciales más grandes

### IBM Granite 4: El Modelo de Lenguaje

Granite 4 es la última generación de modelos de IBM, lanzada en octubre 2025. Tiene una arquitectura híbrida innovadora que combina capas Mamba-2 con capas Transformer tradicionales.

**Características principales:**
- **Eficiencia de memoria:** Reduce el uso de RAM en más del 70% comparado con modelos transformer puros
- **Multilingüe:** Soporta español, inglés, francés, alemán, japonés, portugués, árabe, chino, entre otros
- **Contexto largo:** Puede procesar hasta 512K tokens
- **Optimizado para empresas:** Excelente en seguimiento de instrucciones y function calling

**Tamaños disponibles:**
- **granite4:micro** (3B parámetros) - Ideal para laptops, muy rápido
- **granite4:tiny** (7B MoE, ~1B activos) - Balance entre velocidad y calidad
- **granite4:latest** (32B MoE, ~9B activos) - Máxima calidad, requiere más recursos

---

## Prerrequisitos

### Requisitos de Software

#### 1. Docker Desktop

**Para Windows o Mac:**
1. Descargar desde: https://www.docker.com/products/docker-desktop
2. Instalar siguiendo el asistente
3. Reiniciar tu computadora
4. Verificar que Docker Desktop está corriendo (icono de ballena en la barra de tareas)

**Para Linux (Ubuntu/Debian):**
```bash
sudo apt update
sudo apt install docker.io docker-compose
sudo systemctl start docker
sudo systemctl enable docker
sudo usermod -aG docker $USER
```

Después de instalar, verificá que funciona:
```bash
docker --version
docker-compose --version
```

Deberías ver algo como:
```
Docker version 24.0.7, build afdd53b
Docker Compose version v2.23.0
```

#### 2. Editor de Texto

Cualquier editor sirve: VSCode (recomendado), Sublime Text, Notepad++, Atom.

#### 3. Terminal/Consola

- **Windows:** PowerShell, CMD o Windows Terminal
- **Mac:** Terminal (incluida en el sistema)
- **Linux:** La terminal de tu distribución

### Requisitos de Hardware

**Mínimos (para granite4:micro):**
- RAM: 8GB
- Espacio en disco: 10GB libres
- Procesador: Cualquier CPU moderna de los últimos 5 años

**Recomendados (para granite4:tiny o latest):**
- RAM: 16GB o más
- Espacio en disco: 20GB libres
- Procesador: CPU multinúcleo o GPU (opcional pero mejora la velocidad)

### Conocimientos Previos Necesarios

- Uso básico de la terminal/línea de comandos
- Conceptos básicos de redes (qué es un puerto, localhost)
- Conocimientos previos de la materia (modelos de lenguaje, tareas de NLP)

---

## Paso 1: Preparar el Proyecto

### Crear el Directorio del Proyecto

Abrí tu terminal y ejecutá los siguientes comandos:

**En Windows (PowerShell o CMD):**
```powershell
# Navegar a donde quieras crear el proyecto (ej: Documentos)
cd C:\Users\TuUsuario\Documents

# Crear directorio
mkdir asistente-nlp
cd asistente-nlp
```

**En Mac o Linux:**
```bash
# Navegar a donde quieras crear el proyecto (ej: Documentos)
cd ~/Documents

# Crear directorio
mkdir asistente-nlp
cd asistente-nlp
```

### Crear el archivo docker-compose.yml

Este archivo es el corazón de nuestro proyecto. Define qué servicios vamos a levantar y cómo van a interactuar entre sí.

Abrí tu editor de texto y creá un archivo llamado `docker-compose.yml` en el directorio `asistente-nlp` con el siguiente contenido:

In [ ]:
# IMPORTANTE: NO ejecutes esta celda. 
# Copia el contenido en un archivo llamado docker-compose.yml

"""
services:
  n8n:
    image: docker.n8n.io/n8nio/n8n
    container_name: n8n
    restart: unless-stopped
    ports:
      - "5678:5678"
    environment:
      - GENERIC_TIMEZONE=America/Argentina/Buenos_Aires
      - TZ=America/Argentina/Buenos_Aires
      - N8N_ENFORCE_SETTINGS_FILE_PERMISSIONS=true
      - N8N_RUNNERS_ENABLED=true
    volumes:
      - n8n_data:/home/node/.n8n
    networks:
      - n8n-network
    depends_on:
      - ollama

  ollama:
    image: ollama/ollama:latest
    container_name: ollama
    restart: unless-stopped
    ports:
      - "11434:11434"
    environment:
      - TZ=America/Argentina/Buenos_Aires
    volumes:
      - ollama_data:/root/.ollama
    networks:
      - n8n-network

volumes:
  n8n_data:
  ollama_data:

networks:
  n8n-network:
    driver: bridge
"""

### Entendiendo el docker-compose.yml

Vamos a analizar cada sección:

#### Servicio n8n

```yaml
n8n:
  image: docker.n8n.io/n8nio/n8n
```
Esto le dice a Docker que descargue la imagen oficial de n8n desde su registro.

```yaml
  ports:
    - "5678:5678"
```
Mapea el puerto 5678 del contenedor al puerto 5678 de tu máquina. Esto significa que vas a poder acceder a n8n en `http://localhost:5678`.

```yaml
  environment:
    - GENERIC_TIMEZONE=America/Argentina/Buenos_Aires
    - TZ=America/Argentina/Buenos_Aires
```
Configura la zona horaria. Podés cambiarla a la tuya si no estás en Argentina.

```yaml
  volumes:
    - n8n_data:/home/node/.n8n
```
Esto es crucial. Un **volumen** es un espacio de almacenamiento persistente. Los contenedores Docker son efímeros: si los borrás, perdés todo lo que estaba adentro. Los volúmenes permiten guardar datos importantes (como tus workflows de n8n) que sobreviven aunque elimines el contenedor.

```yaml
  networks:
    - n8n-network
```
Conecta este contenedor a una red privada llamada `n8n-network`. Esto permite que n8n y Ollama se comuniquen entre sí.

```yaml
  depends_on:
    - ollama
```
Le indica a Docker que levante primero Ollama y después n8n.

#### Servicio Ollama

La configuración es similar. Cosas importantes:

```yaml
  ports:
    - "11434:11434"
```
Ollama expone su API en el puerto 11434.

```yaml
  volumes:
    - ollama_data:/root/.ollama
```
Acá se guardan los modelos descargados. Si descargás Granite 4 (que puede pesar varios GB), querés que se guarde en un volumen persistente para no tener que descargarlo de nuevo cada vez.

#### Sección de Volúmenes y Redes

```yaml
volumes:
  n8n_data:
  ollama_data:
```
Declara los volúmenes que vamos a usar.

```yaml
networks:
  n8n-network:
    driver: bridge
```
Crea una red tipo "bridge" (puente) que conecta los contenedores. Dentro de esta red, los contenedores pueden referirse entre sí por su nombre (ej: n8n puede hacer requests a `http://ollama:11434`).

---

## Paso 2: Levantar los Servicios

### Verificar Docker

Antes de continuar, asegurate de que Docker Desktop esté corriendo. Deberías ver el ícono de la ballena en tu barra de tareas.

En tu terminal, ejecutá:
```bash
docker ps
```

Si ves una tabla (aunque esté vacía), Docker está funcionando. Si ves un error, revisá que Docker Desktop esté iniciado.

### Levantar los Contenedores

Desde el directorio donde está tu `docker-compose.yml`, ejecutá:

```bash
docker-compose up -d
```

**¿Qué hace este comando?**
- `docker-compose up`: Levanta los servicios definidos en docker-compose.yml
- `-d`: "Detached mode" - corre los contenedores en segundo plano

**¿Qué va a pasar?**

La primera vez que ejecutes este comando:

1. Docker va a descargar las imágenes de n8n y Ollama (esto puede tardar varios minutos según tu conexión)
2. Va a crear los volúmenes `n8n_data` y `ollama_data`
3. Va a crear la red `n8n-network`
4. Va a iniciar ambos contenedores

Vas a ver una salida similar a esta:
```
[+] Running 5/5
 ✔ Network asistente-nlp_n8n-network  Created
 ✔ Volume "asistente-nlp_n8n_data"    Created
 ✔ Volume "asistente-nlp_ollama_data" Created
 ✔ Container ollama                   Started
 ✔ Container n8n                      Started
```

### Verificar que los Servicios Están Corriendo

```bash
docker-compose ps
```

Deberías ver:
```
NAME      IMAGE                       STATUS         PORTS
n8n       docker.n8n.io/n8nio/n8n    Up 2 minutes   0.0.0.0:5678->5678/tcp
ollama    ollama/ollama:latest        Up 2 minutes   0.0.0.0:11434->11434/tcp
```

La columna STATUS debe decir "Up" para ambos servicios.

### Ver los Logs (Útil para debugging)

Si querés ver qué están haciendo los contenedores:

```bash
# Ver logs de ambos servicios
docker-compose logs -f

# Ver logs solo de n8n
docker-compose logs -f n8n

# Ver logs solo de Ollama
docker-compose logs -f ollama
```

Presioná `Ctrl + C` para salir de los logs.

### Comandos Útiles de Docker Compose

Para referencia futura:

```bash
# Detener servicios (mantiene contenedores y datos)
docker-compose stop

# Iniciar servicios previamente detenidos
docker-compose start

# Reiniciar servicios
docker-compose restart

# Detener y eliminar contenedores (mantiene volúmenes)
docker-compose down

# CUIDADO: Eliminar TODO incluyendo volúmenes (perdés tus datos)
docker-compose down -v
```

---

## Paso 3: Configurar Ollama y Descargar el Modelo

### Verificar que Ollama está Funcionando

Primero, verificá que Ollama esté respondiendo:

```bash
curl http://localhost:11434
```

Deberías ver: `Ollama is running`

Si no tenés `curl` instalado en Windows, podés abrir `http://localhost:11434` en tu navegador y deberías ver el mismo mensaje.

### Descargar el Modelo IBM Granite 4

Ahora viene una decisión importante: elegir qué tamaño de modelo descargar. Esto depende de tu hardware.

#### Opción A: granite4:micro (Recomendado para empezar)

**Características:**
- Tamaño: ~1.7GB
- RAM necesaria: 4-6GB
- Velocidad: Muy rápida
- Ideal para: Laptops con recursos limitados, pruebas rápidas

```bash
docker exec -it ollama ollama pull granite4:micro
```

#### Opción B: granite4:tiny

**Características:**
- Tamaño: ~4GB
- RAM necesaria: 8GB
- Velocidad: Balanceada
- Ideal para: PCs de escritorio con recursos moderados

```bash
docker exec -it ollama ollama pull granite4:tiny
```

#### Opción C: granite4:latest

**Características:**
- Tamaño: ~18GB
- RAM necesaria: 16GB o más
- Velocidad: Más lenta, pero mejor calidad
- Ideal para: Workstations potentes, cuando necesitás la mejor calidad

```bash
docker exec -it ollama ollama pull granite4:latest
```

### Entendiendo el Comando

```bash
docker exec -it ollama ollama pull granite4:micro
```

- `docker exec`: Ejecuta un comando dentro de un contenedor que ya está corriendo
- `-it`: Modo interactivo con terminal (para ver el progreso de la descarga)
- `ollama`: Nombre del contenedor donde queremos ejecutar el comando
- `ollama pull granite4:micro`: El comando específico de Ollama para descargar un modelo

La descarga puede tardar varios minutos. Vas a ver una barra de progreso.

### Verificar que el Modelo se Descargó

```bash
docker exec -it ollama ollama list
```

Deberías ver algo como:
```
NAME                  ID              SIZE      MODIFIED
granite4:micro        abc123def456    1.7 GB    2 minutes ago
```

### Probar el Modelo Interactivamente

Antes de integrarlo con n8n, es buena idea probarlo para verificar que funciona:

```bash
docker exec -it ollama ollama run granite4:micro
```

Esto abre una sesión interactiva. Podés escribir consultas y el modelo va a responder. Por ejemplo, probá:

```
>>> Hola, ¿cómo estás?
```

o

```
>>> Extrae las entidades nombradas de: María García trabaja en Microsoft Argentina
```

Para salir de la sesión interactiva, escribí:
```
/bye
```

### Comandos Útiles de Ollama

```bash
# Listar modelos instalados
docker exec -it ollama ollama list

# Ver información detallada de un modelo
docker exec -it ollama ollama show granite4:micro

# Eliminar un modelo (para liberar espacio)
docker exec -it ollama ollama rm granite4:micro
```

---

## Paso 4: Configuración Inicial de n8n

### Acceder a n8n

Abrí tu navegador web y andá a:

```
http://localhost:5678
```

### Primera Configuración

Si es la primera vez que accedés a n8n, vas a ver una pantalla de bienvenida. 

1. **Email:** Ingresá un email (puede ser cualquiera, es solo para tu sesión local)
2. **First name y Last name:** Tu nombre
3. **Password:** Elegí una contraseña segura
4. Click en **"Get Started"**

**Nota:** Esta configuración se guarda en el volumen Docker `n8n_data`. Si eliminás ese volumen, vas a perder tu usuario y tus workflows.

### Interfaz de n8n

Vas a ver el dashboard de n8n con:
- **Workflows:** Lista de tus workflows (vacía por ahora)
- **Credentials:** Donde guardás claves API y configuraciones
- **Executions:** Historial de ejecuciones de workflows

---

## Paso 5: Construir el Workflow de NLP

Ahora viene la parte central del proyecto: construir el workflow que va a procesar las solicitudes de NLP. En lugar de importar un archivo JSON, vamos a construirlo paso a paso para que entiendas cómo funciona cada componente.

### Crear un Nuevo Workflow

1. En n8n, click en el botón **"+ Add workflow"** (esquina superior derecha)
2. Dale un nombre al workflow: **"Asistente NLP"**
3. Vas a ver un canvas en blanco donde vamos a armar el flujo

### Arquitectura del Workflow

Nuestro workflow va a tener 5 nodos conectados en secuencia:

```
1. Webhook           →    Recibe solicitudes HTTP POST
2. Extraer Mensaje   →    Extrae el campo "mensaje" del JSON
3. Detectar Tarea    →    Identifica qué tarea de NLP hacer
4. Ollama            →    Envía el prompt al modelo
5. Formatear         →    Prepara la respuesta final
```

### Nodo 1: Webhook (Punto de Entrada)

El webhook es el punto de entrada de nuestro sistema. Va a recibir solicitudes HTTP POST con las consultas del usuario.

**Pasos:**

1. Click en **"+ Add first step"** (en el canvas vacío)
2. En el buscador, escribí **"Webhook"**
3. Seleccioná **"Webhook"** de la categoría "Core Nodes"
4. Configurá el nodo:
   - **HTTP Method:** POST
   - **Path:** asistente-nlp
   - **Response Mode:** Last Node
   - Dejá el resto como está

**¿Qué hace este nodo?**

Crea un endpoint HTTP en: `http://localhost:5678/webhook/asistente-nlp`

Cuando alguien envía un POST a esa URL con un JSON como:
```json
{"mensaje": "Extrae entidades de: María vive en Buenos Aires"}
```

El webhook lo recibe y pasa los datos al siguiente nodo.

### Nodo 2: Extraer Mensaje

Este nodo va a extraer el campo "mensaje" del JSON recibido y también va a agregar un timestamp.

**Pasos:**

1. Hover sobre el webhook y click en el **+** que aparece a la derecha
2. Buscá y seleccioná **"Edit Fields (Set)"**
3. Cambiá el nombre del nodo a: **"Extraer Mensaje"**
4. En **"Fields to Set"**, agregá dos campos:

   **Campo 1:**
   - **Name:** mensaje_usuario
   - **Type:** String
   - **Value:** `{{ $json.body.mensaje || '' }}`
   
   **Campo 2:**
   - **Name:** timestamp
   - **Type:** String  
   - **Value:** `{{ $now.toISO() }}`

**¿Qué hace este nodo?**

- Extrae el campo `mensaje` del body del request
- Crea un timestamp con la hora actual
- Pasa estos datos al siguiente nodo

La sintaxis `{{ }}` es la forma de acceder a datos en n8n. Es similar a templates en otros frameworks.

### Nodo 3: Detectar Tarea NLP

Este es el cerebro de nuestro sistema. Va a analizar el mensaje del usuario y decidir qué tipo de tarea de NLP quiere realizar, construyendo un prompt especializado.

**Pasos:**

1. Click en el **+** después de "Extraer Mensaje"
2. Buscá y seleccioná **"Code"**
3. Cambiá el nombre a: **"Detectar Tarea NLP"**
4. Seleccioná **"Run Once for All Items"** en Mode
5. En el editor de código JavaScript, pegá el siguiente código:

In [ ]:
# Código JavaScript para el nodo "Detectar Tarea NLP"
# Copiar y pegar en el editor de n8n

"""
const inputData = $input.first().json;
const mensaje = (inputData.mensaje_usuario || '').toString().toLowerCase();
const timestamp = inputData.timestamp;

let prompt_final = "";
let tarea_detectada = "general";
let instrucciones = "";

// Detección de tarea NER
if (mensaje.includes("ner") || mensaje.includes("entidades") || mensaje.includes("nombres")) {
  tarea_detectada = "NER";
  instrucciones = "Extrae todas las entidades nombradas y clasifícalas.";
  prompt_final = `Eres un experto en Named Entity Recognition (NER).

Tarea: Extrae TODAS las entidades nombradas del texto y clasifícalas en:
- PERSONA: nombres de personas
- ORGANIZACIÓN: empresas, instituciones
- LUGAR: ciudades, países, lugares
- FECHA: fechas, períodos temporales
- CANTIDAD: números, porcentajes, montos

Formato de respuesta:
Entidad | Tipo | Contexto

Texto a analizar:
${mensaje}`;

// Detección de análisis de sentimiento
} else if (mensaje.includes("sentimiento") || mensaje.includes("sentiment") || mensaje.includes("emoción")) {
  tarea_detectada = "Análisis de Sentimiento";
  instrucciones = "Analiza el sentimiento del texto.";
  prompt_final = `Eres un experto en Análisis de Sentimiento.

Tarea: Analiza el sentimiento del siguiente texto.

Clasifícalo como:
- POSITIVO
- NEGATIVO
- NEUTRAL

Proporciona:
1. Clasificación del sentimiento
2. Nivel de confianza (0-100%)
3. Palabras clave que indican el sentimiento
4. Explicación breve

Texto a analizar:
${mensaje}`;

// Detección de sumarización
} else if (mensaje.includes("resume") || mensaje.includes("resumen") || mensaje.includes("sumari")) {
  tarea_detectada = "Sumarización";
  instrucciones = "Resume el texto en puntos clave.";
  prompt_final = `Eres un experto en Sumarización de textos.

Tarea: Resume el siguiente texto de forma concisa.

Proporciona:
1. Resumen en 3-5 puntos clave
2. Idea principal
3. Longitud aproximada: 20% del texto original

Texto a analizar:
${mensaje}`;

// Detección de clasificación
} else if (mensaje.includes("clasifica") || mensaje.includes("categoría") || mensaje.includes("tipo de texto")) {
  tarea_detectada = "Clasificación";
  instrucciones = "Clasifica el tipo y tema del texto.";
  prompt_final = `Eres un experto en Clasificación de Textos.

Tarea: Clasifica el siguiente texto.

Proporciona:
1. Tipo de texto (noticia, opinión, técnico, narrativo, etc.)
2. Tema principal
3. Temas secundarios
4. Nivel de formalidad (formal/informal)
5. Audiencia objetivo

Texto a analizar:
${mensaje}`;

// Detección de extracción de keywords
} else if (mensaje.includes("palabras clave") || mensaje.includes("keywords") || mensaje.includes("términos importantes")) {
  tarea_detectada = "Extracción de Keywords";
  instrucciones = "Extrae las palabras clave más importantes.";
  prompt_final = `Eres un experto en Extracción de Palabras Clave.

Tarea: Extrae las palabras clave más relevantes del texto.

Proporciona:
1. Top 10 palabras clave
2. Relevancia de cada una (Alta/Media/Baja)
3. Bigramas importantes (frases de 2 palabras)
4. Tema central

Texto a analizar:
${mensaje}`;

// Ayuda
} else if (mensaje.includes("ayuda") || mensaje.includes("help") || mensaje.includes("qué puedes hacer")) {
  tarea_detectada = "Ayuda";
  prompt_final = `ASISTENTE NLP - TAREAS DISPONIBLES

Puedo ayudarte con las siguientes tareas de Procesamiento de Lenguaje Natural:

1. NER (Named Entity Recognition)
   Comando: "Extrae entidades de: [tu texto]"
   Ejemplo: "Extrae entidades de: Juan viajó a Madrid en 2024"

2. Análisis de Sentimiento
   Comando: "Analiza el sentimiento de: [tu texto]"
   Ejemplo: "Analiza el sentimiento de: Me encanta este producto"

3. Sumarización
   Comando: "Resume: [tu texto]"
   Ejemplo: "Resume: [texto largo]"

4. Clasificación de Texto
   Comando: "Clasifica: [tu texto]"
   Ejemplo: "Clasifica: Este artículo habla sobre inteligencia artificial"

5. Extracción de Palabras Clave
   Comando: "Palabras clave de: [tu texto]"
   Ejemplo: "Palabras clave de: [tu texto]"

Tip: Simplemente escribe tu solicitud de forma natural`;

// Caso general
} else {
  tarea_detectada = "Consulta General";
  instrucciones = "Responde la consulta del usuario.";
  prompt_final = mensaje;
}

// Retornar datos estructurados
return [{
  json: {
    prompt: prompt_final,
    tarea: tarea_detectada,
    instrucciones: instrucciones,
    mensaje_original: mensaje,
    timestamp: timestamp
  }
}];
"""

**¿Qué hace este código?**

1. Lee el mensaje del usuario
2. Lo convierte a minúsculas para facilitar la detección
3. Busca palabras clave ("ner", "sentimiento", "resume", etc.)
4. Según las palabras encontradas, construye un prompt especializado para esa tarea
5. Retorna un objeto con el prompt, el tipo de tarea detectada y metadata

Por ejemplo, si el usuario escribe:
```
Extrae entidades de: María trabaja en Google
```

El código detecta "entidades" y construye un prompt especializado que le pide al modelo que actúe como experto en NER.

### Nodo 4: Ollama (Conexión con el Modelo)

Este nodo envía el prompt al modelo de lenguaje y recibe la respuesta.

**Pasos:**

1. Click en el **+** después de "Detectar Tarea NLP"
2. Buscá y seleccioná **"HTTP Request"**
3. Cambiá el nombre a: **"Ollama"**
4. Configurá:
   - **Method:** POST
   - **URL:** `http://ollama:11434/api/generate`
   - **Authentication:** None
   - **Send Body:** Activado
   - **Body Content Type:** JSON
   - **Specify Body:** Using JSON
5. En el campo **JSON**, pegá:

In [ ]:
# JSON para el nodo Ollama
# Copiar en el campo JSON del HTTP Request

"""
{
  "model": "granite4:micro",
  "prompt": "={{ $json.prompt }}",
  "stream": false,
  "temperature": 0.7
}
"""

**Importante:** Si descargaste otro tamaño de modelo (tiny o latest), cambiá `"granite4:micro"` por el que descargaste.

**¿Qué hace este nodo?**

Envía un request POST a la API de Ollama con:
- **model:** El modelo a usar
- **prompt:** El prompt construido por el nodo anterior (accede con `$json.prompt`)
- **stream:** false (queremos la respuesta completa, no por streaming)
- **temperature:** 0.7 (controla la creatividad del modelo, 0=determinista, 1=creativo)

**¿Por qué `http://ollama:11434` y no `http://localhost:11434`?**

Dentro de la red Docker, los contenedores se refieren entre sí por su nombre. n8n está en un contenedor y Ollama en otro. Ambos están en la red `n8n-network`, entonces n8n puede acceder a Ollama usando `http://ollama:11434`.

### Nodo 5: Formatear Respuesta

Este nodo final formatea la respuesta para devolver al usuario.

**Pasos:**

1. Click en el **+** después de "Ollama"
2. Seleccioná **"Edit Fields (Set)"**
3. Cambiá el nombre a: **"Formatear Respuesta"**
4. Agregá los siguientes campos:

   **Campo 1:**
   - **Name:** respuesta
   - **Type:** String
   - **Value:** `{{ $json.response }}`
   
   **Campo 2:**
   - **Name:** tarea_realizada
   - **Type:** String
   - **Value:** `{{ $('Detectar Tarea NLP').item.json.tarea }}`
   
   **Campo 3:**
   - **Name:** mensaje_original
   - **Type:** String
   - **Value:** `{{ $('Detectar Tarea NLP').item.json.mensaje_original }}`
   
   **Campo 4:**
   - **Name:** timestamp
   - **Type:** String
   - **Value:** `{{ $now.toISO() }}`

**¿Qué hace este nodo?**

Crea un objeto JSON estructurado con:
- La respuesta del modelo
- Qué tipo de tarea se realizó
- El mensaje original del usuario
- Un timestamp de cuándo se completó

Este es el formato que se devuelve al usuario.

### Guardar y Activar el Workflow

1. Click en **"Save"** (esquina superior derecha)
2. **MUY IMPORTANTE:** Activá el workflow con el toggle **"Active"** (debe ponerse en verde/azul)

Si no activás el workflow, no va a responder a las solicitudes.

### Probar el Workflow desde n8n

Antes de probar desde fuera, podés probar directamente en n8n:

1. Click en el nodo **"Webhook"**
2. Click en **"Listen for Test Event"**
3. En otra pestaña del navegador, o desde tu terminal, enviá un request:

```bash
curl -X POST http://localhost:5678/webhook/asistente-nlp \
  -H "Content-Type: application/json" \
  -d '{"mensaje": "Extrae entidades de: María trabaja en Google en Buenos Aires"}'
```

4. Vas a ver que n8n recibe el request y podés hacer click en **"Execute Workflow"** para ejecutar todo el flujo
5. Observá cómo los datos van pasando por cada nodo

Esto te permite debuggear y ver exactamente qué está pasando en cada paso.

---

## Paso 6: Crear la Interfaz de Usuario

Ahora que tenés el workflow funcionando, necesitás una forma para que los usuarios interactúen con él. Vamos a crear una interfaz web simple.

### Opción A: Interfaz HTML Básica

Creá un archivo `index.html` en tu directorio del proyecto:

In [ ]:
# Contenido del archivo index.html
# Guardar en tu directorio del proyecto

"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Asistente NLP</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 800px;
            margin: 50px auto;
            padding: 20px;
            background-color: #f5f5f5;
        }
        h1 {
            color: #333;
            text-align: center;
        }
        .info-box {
            background-color: #fff;
            border: 1px solid #ddd;
            border-radius: 5px;
            padding: 20px;
            margin-bottom: 20px;
        }
        textarea {
            width: 100%;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 5px;
            font-size: 14px;
            font-family: Arial, sans-serif;
            resize: vertical;
        }
        button {
            background-color: #007bff;
            color: white;
            padding: 12px 30px;
            border: none;
            border-radius: 5px;
            font-size: 16px;
            cursor: pointer;
            display: block;
            margin: 20px auto;
        }
        button:hover {
            background-color: #0056b3;
        }
        #respuesta {
            background-color: #f9f9f9;
        }
        .example {
            color: #666;
            font-size: 13px;
            margin: 5px 0;
        }
    </style>
</head>
<body>
    <h1>Asistente de Procesamiento de Lenguaje Natural</h1>
    
    <div class="info-box">
        <h3>¿Cómo usar el asistente?</h3>
        <p>Iniciá tu mensaje con alguna de estas palabras clave:</p>
        <ul>
            <li><strong>"ner"</strong> o <strong>"entidades"</strong> → Extracción de entidades nombradas</li>
            <li><strong>"sentimiento"</strong> → Análisis de sentimiento</li>
            <li><strong>"resumen"</strong> o <strong>"resume"</strong> → Sumarización</li>
            <li><strong>"clasifica"</strong> → Clasificación de texto</li>
            <li><strong>"palabras clave"</strong> → Extracción de keywords</li>
            <li><strong>"ayuda"</strong> → Ver información completa</li>
        </ul>
        <p class="example"><strong>Ejemplo:</strong> Extrae entidades de: María García trabaja en Microsoft Argentina</p>
    </div>

    <div>
        <h3>Tu consulta:</h3>
        <textarea id="mensaje" rows="6" placeholder="Escribí acá tu consulta o texto para analizar..."></textarea>
    </div>

    <button onclick="enviar()">Enviar</button>

    <div>
        <h3>Respuesta del asistente:</h3>
        <textarea id="respuesta" rows="15" readonly></textarea>
    </div>

    <p style="text-align: center; color: #666; margin-top: 30px;">
        Asistente NLP - Procesamiento de Lenguaje Natural con n8n + Ollama
    </p>

    <script>
        async function enviar() {
            const mensaje = document.getElementById('mensaje').value;
            const respuestaTextarea = document.getElementById('respuesta');

            if (!mensaje.trim()) {
                respuestaTextarea.value = 'Por favor, escribí tu consulta antes de enviar.';
                return;
            }

            respuestaTextarea.value = 'Procesando tu consulta...\n\nEsto puede tomar unos segundos.';

            try {
                const response = await fetch('http://localhost:5678/webhook/asistente-nlp', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ mensaje: mensaje })
                });

                if (!response.ok) {
                    throw new Error(`HTTP ${response.status}: ${await response.text()}`);
                }
                
                const data = await response.json();
                
                let output = '';
                if (data.respuesta) {
                    output += data.respuesta;
                } else {
                    output = JSON.stringify(data, null, 2);
                }
                
                if (data.tarea_realizada) {
                    output += '\n\n─────────────────────────────\n';
                    output += 'Tarea: ' + data.tarea_realizada + '\n';
                    output += 'Timestamp: ' + data.timestamp;
                }
                
                respuestaTextarea.value = output;
            } catch (error) {
                respuestaTextarea.value = 'Error al procesar la solicitud:\n\n' + error.message + 
                    '\n\nVerificá que el workflow esté activo en n8n y que Ollama esté ejecutándose.';
            }
        }

        // Permitir enviar con Ctrl+Enter
        document.getElementById('mensaje').addEventListener('keydown', function(e) {
            if (e.ctrlKey && e.key === 'Enter') {
                enviar();
            }
        });
    </script>
</body>
</html>
"""

Para usar esta interfaz:

1. Abrí el archivo `index.html` con tu navegador (doble click o arrastrar al navegador)
2. Escribí tu consulta
3. Click en "Enviar"

### Opción B: Interfaz con Streamlit (Más Avanzada)

Ya conocés Streamlit de clases anteriores. Podés crear una interfaz más profesional.

Creá un archivo `app_streamlit.py`:

In [ ]:
# app_streamlit.py
import streamlit as st
import requests
from datetime import datetime

st.set_page_config(
    page_title="Asistente NLP",
    page_icon="🤖",
    layout="wide"
)

st.title("Asistente de Procesamiento de Lenguaje Natural")
st.markdown("---")

# Sidebar con información
with st.sidebar:
    st.header("Tareas Disponibles")
    st.markdown("""
    - **NER**: Extracción de entidades
    - **Sentimiento**: Análisis emocional
    - **Resumen**: Sumarización de textos
    - **Clasificación**: Tipo y tema
    - **Keywords**: Palabras clave
    """)
    
    st.markdown("---")
    st.markdown("**Ejemplos:**")
    st.code('Extrae entidades de: Juan vive en Madrid')
    st.code('Analiza el sentimiento de: Me encantó')

# Área principal
mensaje = st.text_area(
    "Tu consulta:",
    height=150,
    placeholder="Escribí acá tu consulta o texto para analizar..."
)

if st.button("Analizar", use_container_width=True, type="primary"):
    if mensaje.strip():
        with st.spinner('Procesando tu consulta...'):
            try:
                response = requests.post(
                    "http://localhost:5678/webhook/asistente-nlp",
                    json={"mensaje": mensaje},
                    timeout=60
                )
                
                if response.status_code == 200:
                    data = response.json()
                    
                    st.success("Análisis completado")
                    
                    st.markdown("### Resultado:")
                    st.markdown(data.get('respuesta', 'Sin respuesta'))
                    
                    st.markdown("---")
                    col1, col2 = st.columns(2)
                    with col1:
                        st.metric("Tarea", data.get('tarea_realizada', 'N/A'))
                    with col2:
                        timestamp = data.get('timestamp', '')
                        if timestamp:
                            dt = datetime.fromisoformat(timestamp.replace('Z', '+00:00'))
                            st.metric("Hora", dt.strftime('%H:%M:%S'))
                else:
                    st.error(f"Error HTTP {response.status_code}")
                    st.code(response.text)
                    
            except requests.exceptions.ConnectionError:
                st.error("No se pudo conectar con n8n. Verificá que esté corriendo.")
            except Exception as e:
                st.error(f"Error: {str(e)}")
    else:
        st.warning("Por favor, escribí tu consulta antes de enviar.")

st.markdown("---")
st.caption("Asistente NLP - n8n + Ollama + Granite 4")

Para ejecutar la app Streamlit:

```bash
# Instalar Streamlit (si no lo tenés)
pip install streamlit requests

# Ejecutar
streamlit run app_streamlit.py
```

La app se va a abrir en `http://localhost:8501`

### Pruebas Sugeridas

Probá tu asistente con estas consultas:

**1. NER:**
```
Extrae entidades de: María García trabaja en Microsoft Argentina en Buenos Aires desde marzo de 2023
```

**2. Sentimiento:**
```
Analiza el sentimiento de: El curso me pareció excelente, aprendí muchísimo y los profesores fueron muy claros en sus explicaciones
```

**3. Resumen:**
```
Resume: Docker es una plataforma de containerización que permite empaquetar aplicaciones con todas sus dependencias. Los contenedores son más livianos que las máquinas virtuales porque comparten el kernel del sistema operativo. Docker Compose facilita la orquestación de múltiples contenedores, permitiendo definir toda la infraestructura en un archivo YAML.
```

**4. Clasificación:**
```
Clasifica: La inteligencia artificial está revolucionando el procesamiento del lenguaje natural mediante el uso de modelos de lenguaje grandes
```

**5. Keywords:**
```
Palabras clave de: El procesamiento del lenguaje natural es una rama de la inteligencia artificial que se enfoca en la interacción entre computadoras y humanos usando lenguaje natural
```

**6. Ayuda:**
```
ayuda
```

---

## Paso 7: Crear una Imagen Docker Personalizada

Hasta ahora usamos imágenes pre-construidas (n8n y Ollama). Ahora vas a aprender a crear tu propia imagen Docker que contenga tu aplicación Streamlit.

### ¿Por qué crear una imagen?

- **Portabilidad:** Podés compartir tu aplicación completa, no solo el código
- **Reproducibilidad:** La imagen incluye todas las dependencias exactas
- **Despliegue:** Facilita el deployment en servidores
- **Versionado:** Podés tener múltiples versiones (v1.0, v1.1, v2.0)

### Crear el Dockerfile

Un Dockerfile es un archivo de texto con instrucciones para construir una imagen.

Creá un archivo llamado `Dockerfile` (sin extensión) en tu proyecto:

In [ ]:
# Contenido del Dockerfile

"""
# Imagen base: Python 3.11 slim (versión liviana)
FROM python:3.11-slim

# Metadata de la imagen
LABEL maintainer="tu_email@example.com"
LABEL description="Asistente NLP con Streamlit"
LABEL version="1.0"

# Directorio de trabajo dentro del contenedor
WORKDIR /app

# Copiar archivo de dependencias
COPY requirements.txt .

# Instalar dependencias Python
RUN pip install --no-cache-dir -r requirements.txt

# Copiar archivos de la aplicación
COPY app_streamlit.py .

# Exponer puerto de Streamlit
EXPOSE 8501

# Comando para ejecutar la aplicación
CMD ["streamlit", "run", "app_streamlit.py", "--server.address", "0.0.0.0"]
"""

### Crear el archivo requirements.txt

Lista las dependencias Python de tu aplicación:

In [ ]:
# Contenido de requirements.txt

"""
streamlit==1.32.0
requests==2.31.0
"""

### Construir la Imagen

Desde tu directorio del proyecto, ejecutá:

```bash
docker build -t asistente-nlp-frontend:v1.0 .
```

**¿Qué significa cada parte?**
- `docker build`: Comando para construir una imagen
- `-t asistente-nlp-frontend:v1.0`: Tag (nombre y versión) de la imagen
- `.`: Contexto de build (directorio actual)

El proceso puede tardar unos minutos. Docker va a:
1. Descargar la imagen base Python
2. Copiar tu código
3. Instalar las dependencias
4. Crear la imagen final

### Verificar la Imagen

```bash
docker images
```

Deberías ver:
```
REPOSITORY                  TAG       SIZE
asistente-nlp-frontend      v1.0      450MB
```

### Actualizar docker-compose.yml

Agregá tu nueva imagen al docker-compose existente:

In [ ]:
# Agregar al final de docker-compose.yml (después del servicio ollama)

"""
  streamlit-app:
    image: asistente-nlp-frontend:v1.0
    container_name: asistente-frontend
    restart: unless-stopped
    ports:
      - "8501:8501"
    networks:
      - n8n-network
    depends_on:
      - n8n
"""

Ahora reiniciá los servicios:

```bash
docker-compose up -d
```

Tenés tres servicios corriendo:
- **Ollama** en puerto 11434
- **n8n** en puerto 5678  
- **Streamlit** en puerto 8501

### Compartir tu Imagen

**Opción A: Guardar como archivo**

```bash
# Exportar imagen a archivo .tar
docker save asistente-nlp-frontend:v1.0 -o asistente-nlp-frontend.tar

# Comprimir (opcional)
gzip asistente-nlp-frontend.tar
```

Ahora podés compartir el archivo `.tar.gz`. Para importarlo en otra máquina:

```bash
docker load -i asistente-nlp-frontend.tar
```

**Opción B: Subir a Docker Hub**

Docker Hub es un registro público de imágenes (como GitHub para código).

```bash
# 1. Crear cuenta en hub.docker.com (gratis)

# 2. Login desde terminal
docker login

# 3. Taggear con tu usuario
docker tag asistente-nlp-frontend:v1.0 tuusuario/asistente-nlp-frontend:v1.0

# 4. Subir a Docker Hub
docker push tuusuario/asistente-nlp-frontend:v1.0
```

Ahora cualquiera puede descargar tu imagen con:
```bash
docker pull tuusuario/asistente-nlp-frontend:v1.0
```

---

## Paso 8: Despliegue en Servidores

Docker se usa masivamente en servidores y en la nube. Esta sección te introduce a los conceptos de deployment en producción.

### ¿Por qué Desplegar en un Servidor?

Hasta ahora tu aplicación corre en tu máquina local. Para que otros puedan usarla necesitás:

- **Disponibilidad 24/7:** Servidores que no se apagan
- **Acceso público:** Dirección IP pública o dominio
- **Recursos:** Servidores con más capacidad
- **Confiabilidad:** Infraestructura con backups y redundancia

### Docker en Servidores

Docker es extremadamente popular en servidores porque:

1. **Simplicidad:** El mismo `docker-compose.yml` funciona en tu laptop y en el servidor
2. **Aislamiento:** Múltiples aplicaciones pueden correr sin interferir entre sí
3. **Escalabilidad:** Podés replicar contenedores fácilmente
4. **Portabilidad:** Migrás entre providers sin cambiar nada

### Opciones de Hosting

#### Proveedores Cloud Principales

**1. DigitalOcean (Recomendado para empezar)**
- **Droplets:** Servidores virtuales simples
- **Costo:** Desde $6/mes
- **Ventaja:** Muy fácil de usar, buena documentación
- **Ideal para:** Proyectos pequeños, aprendizaje

**2. Amazon Web Services (AWS)**
- **EC2:** Máquinas virtuales con Docker
- **Lightsail:** Opción más simple (similar a DigitalOcean)
- **Costo:** Desde $3.50/mes (Lightsail)
- **Ventaja:** Escalabilidad casi infinita
- **Desventaja:** Curva de aprendizaje empinada

**3. Google Cloud Platform (GCP)**
- **Compute Engine:** VMs con Docker
- **Cloud Run:** Deploy directo de contenedores (serverless)
- **Ventaja:** Integración con servicios de Google

**4. Railway (Recomendado para estudiantes)**
- **Ventaja:** $5 gratis por mes para estudiantes
- **Deploy automático** desde GitHub
- **Ideal para:** Prototipos, proyectos educativos

### Proceso General de Despliegue

El proceso es similar en todos los providers:

**1. Crear un servidor**
```bash
# En el provider elegido:
# - Seleccionar Ubuntu 22.04 LTS
# - Elegir plan (mínimo 2GB RAM)
# - Configurar SSH key
# - Crear servidor
```

**2. Conectarse por SSH**
```bash
ssh root@tu-servidor-ip
```

**3. Instalar Docker**
```bash
# Actualizar sistema
apt update && apt upgrade -y

# Instalar Docker
curl -fsSL https://get.docker.com -o get-docker.sh
sh get-docker.sh

# Instalar Docker Compose
apt install docker-compose -y
```

**4. Subir tu proyecto**
```bash
# Opción A: Clonar desde Git (recomendado)
git clone https://github.com/tuusuario/asistente-nlp.git
cd asistente-nlp

# Opción B: Copiar archivos con SCP desde tu máquina
# scp -r /ruta/local/proyecto root@servidor-ip:/root/
```

**5. Levantar servicios**
```bash
docker-compose up -d
```

**6. Configurar firewall**
```bash
# Permitir puertos necesarios
ufw allow 5678/tcp   # n8n
ufw allow 8501/tcp   # Streamlit
ufw allow 22/tcp     # SSH
ufw enable
```

**7. Acceder a tu aplicación**
```
http://tu-servidor-ip:8501  (Streamlit)
http://tu-servidor-ip:5678  (n8n)
```

### Consideraciones de Producción

Para un deployment serio considerá:

**Seguridad:**
- Cambiar puertos por defecto
- Usar variables de entorno para secrets
- Configurar SSL/TLS (HTTPS)
- Actualizar regularmente
- Configurar fail2ban (protección contra ataques)

**Disponibilidad:**
- Configurar backups automáticos
- Monitoreo de servicios
- Logs centralizados
- Alertas ante caídas

**Rendimiento:**
- Usar un reverse proxy (Nginx)
- Configurar límites de recursos
- Caché cuando sea apropiado
- CDN para contenido estático

### Costos Estimados

Para este proyecto:

**Opción Básica:**
- DigitalOcean Droplet 2GB: $12/mes
- Dominio: $10-15/año
- SSL: Gratis (Let's Encrypt)
- **Total:** ~$13-15/mes

**Opción Estudiante:**
- Railway: $5 crédito/mes (gratis con GitHub Student Pack)
- **Total:** Gratis durante estudios

### Próximos Pasos en Deployment

Para profundizar:

1. **Dominio personalizado:** En lugar de IP, usar `asistente-nlp.tudominio.com`
2. **HTTPS:** Configurar SSL con Let's Encrypt
3. **CI/CD:** Deploy automático cuando hacés push a Git
4. **Kubernetes:** Para proyectos grandes que necesitan escalar
5. **Monitoring:** Grafana + Prometheus para observabilidad

---

## Solución de Problemas Comunes

### Docker no inicia

**Síntoma:** Errores al ejecutar comandos Docker

**Solución:**
1. Verificar que Docker Desktop esté corriendo
2. En Windows, reiniciar el servicio Docker
3. En Linux: `sudo systemctl restart docker`

### Puerto en uso

**Síntoma:** Error "port is already allocated"

**Solución:**
```bash
# Ver qué está usando el puerto (ej: 5678)
# Windows:
netstat -ano | findstr :5678

# Mac/Linux:
lsof -i :5678

# Matar el proceso o cambiar puerto en docker-compose.yml
```

### n8n no se conecta con Ollama

**Síntoma:** Timeout o error de conexión

**Solución:**
1. Verificar que ambos contenedores estén UP: `docker-compose ps`
2. Verificar la URL en el nodo HTTP Request: debe ser `http://ollama:11434` (NO localhost)
3. Verificar que estén en la misma red: `docker network inspect nombreproyecto_n8n-network`

### Modelo muy lento

**Síntoma:** Respuestas tardan minutos

**Solución:**
1. Verificar uso de recursos: `docker stats`
2. Si tenés poca RAM, usar un modelo más chico (granite4:micro)
3. Cerrar otras aplicaciones pesadas
4. Aumentar timeout en el nodo HTTP Request

### Sin espacio en disco

**Síntoma:** "no space left on device"

**Solución:**
```bash
# Limpiar imágenes y contenedores sin usar
docker system prune -a

# Eliminar modelos de Ollama que no uses
docker exec -it ollama ollama rm nombre_modelo
```

### Perdí mis workflows

**Síntoma:** Después de `docker-compose down -v` se perdió todo

**Prevención:**
- NUNCA uses `-v` a menos que quieras borrar datos
- Exportá workflows regularmente desde n8n
- Hacé backups del volumen:
```bash
docker run --rm -v nombreproyecto_n8n_data:/data -v $(pwd):/backup \
  ubuntu tar czf /backup/n8n_backup.tar.gz /data
```

---

## Ejercicios Prácticos

### Ejercicio 1: Experimentación con Modelos

**Objetivo:** Comparar diferentes tamaños de Granite 4

1. Descargá granite4:micro, granite4:tiny
2. Probá la misma consulta con ambos
3. Medí tiempo de respuesta y calidad
4. Documentá tus conclusiones

### Ejercicio 2: Extender el Workflow

**Objetivo:** Agregar nueva tarea de NLP

Implementá soporte para **Corrección gramatical**:
1. Editá el nodo "Detectar Tarea NLP"
2. Agregá detección para "corrige" o "corrección"
3. Creá un prompt especializado
4. Probá con texto con errores

### Ejercicio 3: Mejorar la Interfaz

**Objetivo:** Agregar funcionalidades a Streamlit

1. Implementá historial de consultas
2. Agregá estadísticas de uso
3. Permití exportar resultados a TXT/JSON

### Ejercicio 4: Despliegue Local Completo

**Objetivo:** Crear sistema completamente funcional

1. Creá tu imagen Docker personalizada
2. Actualizá docker-compose.yml con todos los servicios
3. Documentá el proceso de instalación
4. Compartí la imagen con un compañero

### Proyecto Final: Sistema Multiagente

**Objetivo:** Implementar arquitectura avanzada

Creá múltiples workflows especializados:
1. Workflow coordinador que decide qué workflow llamar
2. Workflow especializado en NER
3. Workflow especializado en análisis de sentimiento
4. Workflow que combina resultados de múltiples análisis

---

## Comandos de Referencia Rápida

### Docker Compose
```bash
docker-compose up -d          # Levantar servicios
docker-compose down           # Detener y eliminar contenedores
docker-compose ps             # Ver estado de servicios
docker-compose logs -f        # Ver logs en tiempo real
docker-compose restart        # Reiniciar servicios
```

### Docker Básico
```bash
docker ps                     # Contenedores corriendo
docker ps -a                  # Todos los contenedores
docker images                 # Listar imágenes
docker logs nombre            # Ver logs de un contenedor
docker exec -it nombre bash   # Entrar a un contenedor
docker stats                  # Ver uso de recursos
```

### Ollama
```bash
docker exec -it ollama ollama list         # Listar modelos
docker exec -it ollama ollama pull modelo  # Descargar modelo
docker exec -it ollama ollama rm modelo    # Eliminar modelo
docker exec -it ollama ollama run modelo   # Ejecutar interactivamente
```

### Limpieza
```bash
docker system prune           # Limpiar recursos sin usar
docker system prune -a        # Limpiar TODO sin usar
docker volume prune           # Limpiar volúmenes sin usar
```

---

## Conclusiones y Próximos Pasos

### Lo que Aprendiste

En este laboratorio desarrollaste habilidades fundamentales:

1. **Fundamentos de Docker:** Contenedores, imágenes, volúmenes, redes
2. **Docker Compose:** Orquestación de servicios multi-contenedor
3. **n8n:** Automatización visual de workflows
4. **Ollama:** Ejecución local de modelos de lenguaje
5. **Granite 4:** Modelo de lenguaje híbrido de última generación
6. **Construcción de imágenes:** Dockerfiles y distribución
7. **Conceptos de deployment:** Despliegue en servidores

### Aplicabilidad Profesional

Estas habilidades son altamente demandadas en:
- DevOps Engineer
- MLOps Engineer
- Backend Developer
- Data Engineer
- NLP Engineer

### Recursos para Profundizar

**Docker:**
- Documentación oficial: https://docs.docker.com/
- Docker Curriculum: https://docker-curriculum.com/

**n8n:**
- Documentación: https://docs.n8n.io/
- Templates de workflows: https://n8n.io/workflows/

**Ollama:**
- Catálogo de modelos: https://ollama.ai/library
- GitHub: https://github.com/ollama/ollama

### Preguntas de Reflexión

Antes de finalizar, reflexioná sobre:

1. **LLMs locales vs APIs:** ¿Cuándo preferirías cada opción en un proyecto real?
2. **Arquitectura:** ¿Cómo escalarías este sistema para 100 usuarios simultáneos?
3. **Seguridad:** ¿Qué vulnerabilidades identificás en el sistema actual?
4. **Costos:** ¿Qué sería más económico: Ollama local o usar APIs de OpenAI/Anthropic?

---

**Última actualización:** Noviembre 2025  